# Konfig - Demo



`konfig` is a wrapper around `ml_collection.ConfigDict`.

It allows to turn any Python function call /class construction into a nested dict which can be serialized/deserialized.

This unlocks (among others) type checking and autocomplete inside configuration files.

## Imports



Import `konfig`:

In [10]:
from etils.lazy_imports import *

with ecolab.adhoc():
  from kauldron import konfig

## End-to-end example

Here is a short example demonstrating how `konfig` work:


In [17]:
# Step 1: Import your modules you want to be configurable (can be any module)
with konfig.imports():
  from flax import linen as nn

# Step 2: Create your configuration. Nothing is executed yet.
model_cfg = nn.Dense(features=32)  # This create a `ConfigDict` instead

# Step 3: The config object can be mutated, serialized to json,...
model_cfg.features = 64
model_cfg.use_bias = True

# Step 4: Resolve the config
# This is where the actual Python object are created (here the flax module)
model = konfig.resolve(model_cfg)

## Basic usage

### Create a config (from any Python objects)

Here is a nested Python call that get executed (can be any function, modules,...):

In [23]:
from flax import linen as nn

model = nn.Sequential(layers=[
    nn.Dense(features=32),
    nn.Dropout(rate=0.5),
])

By wrapping the call inside a `with konfig.mock_modules():` contextmanager, the nested call is turned into a nested `konfig.ConfigDict` object, rather than being executed:

In [24]:
with konfig.mock_modules():
  model_cfg = nn.Sequential(layers=[  # model_cfg is a ConfigDict ! Not nn.Sequential!
      nn.Dense(features=32),
      nn.Dropout(rate=0.5),
  ]);

<ConfigDict[nn.Sequential(
    layers=[
        nn.Dense(features=32),
        nn.Dropout(rate=0.5),
    ],
)]>

An equivalent to build the config is to import the module in a `with kontext.imports():` contextmanager:

In [25]:
with konfig.imports():
  from flax import linen as nn

model_cfg = nn.Sequential(layers=[
    nn.Dense(features=32),
    nn.Dropout(rate=0.5),
]);

<ConfigDict[nn.Sequential(
    layers=[
        nn.Dense(features=32),
        nn.Dropout(rate=0.5),
    ],
)]>

To summarize: Use `with kontext.imports():` or `with konfig.mock_modules():` to turn any Python call into configurable.

### Mutate the config

The `ConfigDict` object can be mutated (directly or through command line), like:

In [26]:
model_cfg.layers[0].features = 64
model_cfg.layers[0].use_bias = False
model_cfg.layers[1].rate = 0.9
model_cfg

<ConfigDict[nn.Sequential(
    layers=[
        nn.Dense(
            features=64,
            use_bias=False,
        ),
        nn.Dropout(rate=0.9),
    ],
)]>

### Resolve the config

Once the config has been updated, it can be resolved into the actual object:

In [27]:
model = konfig.resolve(model_cfg);

Sequential(
    # attributes
    layers = (Dense(
        # attributes
        features = 64
        use_bias = False
        dtype = None
        param_dtype = float32
        precision = None
        kernel_init = init
        bias_init = zeros
        dot_general = None
        dot_general_cls = None
    ), Dropout(
        # attributes
        rate = 0.9
        broadcast_dims = ()
        deterministic = None
        rng_collection = 'dropout'
    ))
)

To summarize:

* `model_cfg` is the `ConfigDict` (mutable), created inside the `with konfig.mock_modules():` contextmanager
* `model` is the resolved object (after calling `konfig.resolve`)

In [29]:
assert isinstance(model_cfg, konfig.ConfigDict)
assert isinstance(model, flax.linen.Module)

### Serialize the config

`ConfigDict` objects are just very simple nested dict that are human readable and easily serializable:

In [30]:
serialized_dict = json.loads(model_cfg.to_json());p

{
    '__qualname__': 'flax.linen:Sequential',
    'layers': [
        {
            '__qualname__': 'flax.linen:Dense',
            'features': 64,
            'use_bias': False,
        },
        {
            '__qualname__': 'flax.linen:Dropout',
            'rate': 0.9,
        },
    ],
}


Deserialization done by passing the json dict to `konfig.ConfigDict`:

In [31]:
konfig.ConfigDict(serialized_dict)

<ConfigDict[nn.Sequential(
    layers=[
        nn.Dense(
            features=64,
            use_bias=False,
        ),
        nn.Dropout(rate=0.9),
    ],
)]>

Caveat:

* Json do no preserve tuple/list information. `tuple` are restored as `list`

## Benefits

* Everything is configurable by default (flax, optax, your custom dataclasses,...) without **any** change in your code.
* No duplication between code and the config system (`Model` vs `ModelParams`), as code is directly customizable
* Config files nativelly support type checking and auto-complete. And benefit from all IDE overlay (hover on a class to see the doc, click on a attribute to see where it is defined in the code,...).


## Advanced usage

### Mutating a config

Indidivual attributes can be accessed/modified:

* kwargs - by mutating `cfg.kwarg_name = `:

In [37]:
with konfig.mock_modules():
  model_cfg.layers = [nn.Dense(features=32, dtype=np.float32)]
model_cfg

<ConfigDict[nn.Sequential(
    layers=[
        nn.Dense(
            dtype=np.float32,
            features=32,
        ),
    ],
)]>

* args - by mutating `cfg[arg_id] = `:

In [19]:
with konfig.mock_modules():
  cfg = pathlib.Path('aaa', 'bbb')

cfg[0] = 'root'
cfg[-1] = 'last'
cfg[2] = 'append'  # Append an additional argument
cfg

<ConfigDict[pathlib.Path('root', 'last', 'append')]>

* class/function - by mutating `cfg.__qualname__ =` (to `<import>:<name>`):

In [20]:
cfg.__qualname__ = 'pathlib:PurePath'
cfg

<ConfigDict[pathlib.PurePath('root', 'last', 'append')]>

* constants - by mutating `cfg.__const__ =`

In [38]:
model_cfg.layers[0].dtype.__const__ = 'numpy:float64'